# Chains

- Using LLMs in isolation is fine for many applications.
- But in some cases, it can be useful to use a **Chain**: a sequence of calls to components.


In [1]:
import os
from pprint import pprint
from dotenv import load_dotenv

In [2]:
# 1. Copy .env.example file as .env: `cp .env.example .env`
# 2. Open .env file and set all the env variables
load_dotenv(".env")
OPENAI_KEY = os.getenv("OPENAI_KEY")
assert OPENAI_KEY, "Please set your OPENAI_KEY environment variable."

## LLM Chain
- The `LLMChain` is the simplest chain, but it is also used for most other, more complex, chains.
- It takes in a prompt template, formats it with the user input, and returns the response from an LLM.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [4]:
# The Chat Model
chat_model = ChatOpenAI(
    openai_api_key=OPENAI_KEY,
    model_name="gpt-4",
    temperature=0,
    model_kwargs={"top_p":1},
)

In [5]:
prompt_template = ChatPromptTemplate.from_template(
    "How would you describe the country {country} in a very short (max 4 line) rhyming poem?"
)

In [6]:
chain = LLMChain(llm=chat_model, prompt=prompt_template)

In [7]:
print(chain.run("Switzerland"))

Land of the Alps, so grand and tall,
Switzerland, the fairest of all. 
Where crystal lakes and chalets reside, 
A peaceful haven, world's pride.


## Sequential Chains

- Sequential Chains feed the output of a chain as the input of a following chain.
- `SimpleSequentialChain` — allows for only single input and single output
- `SequentialChain` — allows for multiple inputs and outputs

In [8]:
prompt_template_1 = ChatPromptTemplate.from_template(
    "How would you describe the country {country} in a very short (max 4 line) rhyming poem?"
)

prompt_template_2 = ChatPromptTemplate.from_template(
    "Give a title to this poem: {poem}"
) 

In [9]:
chain_1 = LLMChain(llm=chat_model, prompt=prompt_template_1)
chain_2 = LLMChain(llm=chat_model, prompt=prompt_template_2)

chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)

In [10]:
chain.run("France")



> Entering new SimpleSequentialChain chain...
In France, where romance and art dance,
Fine wine flows, Eiffel Tower's glow enchants.
History's majestic, fashion's quite drastic,
Culture-rich land, its beauty fantastic.
"Ode to the French Elegance"

> Finished chain.


'"Ode to the French Elegance"'

In [11]:
# 1 -> 2 --\
#           |--> 4
#      3 --/

prompt_template_1 = ChatPromptTemplate.from_template(
    "Translate the following into English: {raw_text}"
)

prompt_template_2 = ChatPromptTemplate.from_template(
    "Summarize the following in one sentence: {en_text}"
) 

prompt_template_3 = ChatPromptTemplate.from_template(
    "Say what is the language of the following text: {raw_text}"
)

prompt_template_4 = ChatPromptTemplate.from_template(
    "Expand the following text into bullet points written in {lang}: {summary_en_text}"
)


chain_1 = LLMChain(llm=chat_model, prompt=prompt_template_1, output_key="en_text")
chain_2 = LLMChain(llm=chat_model, prompt=prompt_template_2, output_key="summary_en_text")
chain_3 = LLMChain(llm=chat_model, prompt=prompt_template_3, output_key="lang")
chain_4 = LLMChain(llm=chat_model, prompt=prompt_template_4, output_key="bullets_lang_text")

In [12]:
overall_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["raw_text"],
    output_variables=["en_text", "summary_en_text", "lang", "bullets_lang_text"],
    verbose=True,
)


In [13]:
raw_text = """
La commune de Genève s'est constituée sous sa forme actuelle en 1930, au moment de la fusion des 
communes de Genève (Genève-Cité), de Plainpalais, des Eaux-Vives et du Petit-Saconnex. Un projet 
supprimant la commune et mettant la ville sous la tutelle du canton échoue devant le peuple genevois 
en décembre 1926. Après la fusion, quatre arrondissements (portant les noms des anciennes communes)
sont maintenus jusqu'en 1958, date à laquelle, avec le processus de dépeuplement du centre de la
ville et de déplacement de la population à sa périphérie, ils sont supprimés.

Il apparaît, au début du xxie siècle, qu'une distinction des tâches de la ville et de celles du 
canton n'est toujours pas clairement réalisée. Dans ce contexte, le Conseil d'État propose en 1999 
une fusion entre ville et canton mais la ville, gérée par une majorité de gauche opposée à celle du 
gouvernement genevois, refuse la démarche au nom de l'autonomie communale.

La ville de Genève reste toutefois subdivisée en quatre secteurs : La Cité, Plainpalais, Les 
Eaux-Vives et Le Petit-Saconnex. Alors que l'Office fédéral de la statistique (OFS) recense au 
niveau fédéral les communes en Suisse, c'est l'administration cantonale genevoise qui se charge 
du découpage des communes genevoises (sous-secteurs).
"""

output = overall_chain(raw_text)
for key, value in output.items():
    print(f"{key.upper()}:\n{value}\n")



> Entering new SequentialChain chain...

> Finished chain.
RAW_TEXT:

La commune de Genève s'est constituée sous sa forme actuelle en 1930, au moment de la fusion des 
communes de Genève (Genève-Cité), de Plainpalais, des Eaux-Vives et du Petit-Saconnex. Un projet 
supprimant la commune et mettant la ville sous la tutelle du canton échoue devant le peuple genevois 
en décembre 1926. Après la fusion, quatre arrondissements (portant les noms des anciennes communes)
sont maintenus jusqu'en 1958, date à laquelle, avec le processus de dépeuplement du centre de la
ville et de déplacement de la population à sa périphérie, ils sont supprimés.

Il apparaît, au début du xxie siècle, qu'une distinction des tâches de la ville et de celles du 
canton n'est toujours pas clairement réalisée. Dans ce contexte, le Conseil d'État propose en 1999 
une fusion entre ville et canton mais la ville, gérée par une majorité de gauche opposée à celle du 
gouvernement genevois, refuse la démarche au nom de l'au

## Router Chains

- In many cases, we may want to decide which component to run after a first component based on the output of the first component.
- For instance, a first component could be sentiment analysis.
  - If the sentiment is positive: run a component that uses enthusiastic language to respond.
  - If the sentiment is negative: run a component that uses succinct but empathetic language to respond.
- A `LLMRouterChain` can be used exactly for this purpose.


In [14]:
prompt_template_detection = ChatPromptTemplate.from_template(
    """
    Determine if the content of this message is Positive or Negative.

    {input}
    """
)

prompt_template_positive = ChatPromptTemplate.from_template(
    """
    Write a reply to this message by using an enthusiastic language including emojis.

    {input}
    """
)

prompt_template_negative = ChatPromptTemplate.from_template(
    """
    Write a reply to this message by using a succinct but empathetic language.

    {input}
    """
)

In [15]:
prompt_infos = [
    {
        "name": "positive_reply",
        "description": "Good for replying to positive messages",
        "prompt_template": prompt_template_positive,
    },
    {
        "name": "negative_reply",
        "description": "Good for replying to negative messages",
        "prompt_template": prompt_template_negative,
    }
]

In [16]:
destination_chains = dict()

for p in prompt_infos:
    destination_chains[p["name"]] = LLMChain(llm=chat_model, prompt=p["prompt_template"])
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [17]:
# This will be used when the router cannot decide if it's a Positive or Negative

default_prompt = ChatPromptTemplate.from_template(
    """
    Write a reply to this message in succinct and generic terms.

    {input}
    """
)
    
default_chain = LLMChain(llm=chat_model, prompt=default_prompt)

In [18]:
prompt_template_router = """
Given a message, select the model prompt best suited for replying to it.
You will be given the names of the available prompts and a description of what the prompt is best 
suited for.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input (e.g. fix typos)
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt
names specified below OR it can be "DEFAULT" if the input is not
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json in the beginning and the ``` at the end)>>
"""

In [19]:
router_template = prompt_template_router.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(chat_model, router_prompt)

In [20]:
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain,
    verbose=True,
)

In [21]:
response = chain.run(
    """
    I am so happy today! I just got a new job and I am going to start next week.
    """
)

print(response)



> Entering new MultiPromptChain chain...


/Users/casalegn/dev/LangChain-Experiments/tmp-venv/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


positive_reply: {'input': 'I am so happy today! I just got a new job and I am going to start next week.'}
> Finished chain.
Wow, that's amazing news! 😍 Huge congratulations on your new job! 👏👏 I'm absolutely thrilled for you! 💃🎉 Best of luck for your journey ahead! Let's celebrate this fantastic achievement! 🥳🍾


In [22]:
response = chain.run(
    """
    My dog broke his leg and I am so sad. I don't know what to do.
    """
)

print(response)



> Entering new MultiPromptChain chain...
negative_reply: {'input': "My dog broke his leg and I am so sad. I don't know what to do."}
> Finished chain.
I'm really sorry to hear about your dog. It's understandable you're feeling this way. Please take him to a vet as soon as possible for a professional assessment. Everything will be okay, and remember he needs your strength and love in this challenging time.
